# **Data Cleaning Notebook**

## Objectives

* Further data exploration
* Evaluate missing data
* Decide on imputation methods for missing data
* Split the data into Train and Test sets.

## Inputs

* outputs/datasets/collection/house_prices_records.csv 

## Outputs

* Generate cleaned Train and Test sets, saved under outputs/datasets/cleaned 

## Additional Comments

* CRISP-DM: *Data Understanding/Preparation* 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

Import all necessary packages and libraries for the notebook

In [ ]:
import os
import pandas as pd
from ydata_profiling import ProfileReport
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps
from sklearn.pipeline import Pipeline
from feature_engine.selection import DropFeatures
from feature_engine.imputation import MeanMedianImputer
from feature_engine.imputation import CategoricalImputer
from feature_engine.imputation import ArbitraryNumberImputer
sns.set(style="whitegrid")

---

# Load Collected Data

In [ ]:
df_raw_path = "outputs/datasets/collection/house_prices_records.csv"
df = pd.read_csv(df_raw_path)
df.head(3)

# Data Exploration

Reviewing the list of columns with missing data

In [ ]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

In [ ]:
from pandas_profiling import ProfileReport
if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

Observation:
* in total 9 columns are missing data 
* a total of approx.. 26% of cells are missing data.
* some of the variables with missing values also have too many zeros.

---

## Correlation and PPS Analysis

Custom functions used to calculate the Pearson and Spearman coefficients, and the PPS are taken from Code Institute's "Churnometer" walkthrough project.

In [ ]:
def heatmap_corr(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=bool)
        mask[np.triu_indices_from(mask)] = True
        mask[abs(df) < threshold] = True

        fig, axes = plt.subplots(figsize=figsize)
        sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                    mask=mask, cmap='viridis', annot_kws={"size": font_annot}, ax=axes,
                    linewidth=0.5
                    )
        axes.set_yticklabels(df.columns, rotation=0)
        plt.ylim(len(df.columns), 0)
        plt.show()


def heatmap_pps(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=bool)
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        ax = sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                         mask=mask, cmap='rocket_r', annot_kws={"size": font_annot},
                         linewidth=0.05, linecolor='grey')
        plt.ylim(len(df.columns), 0)
        plt.show()


def CalculateCorrAndPPS(df):
    df_corr_spearman = df.corr(method="spearman")
    df_corr_pearson = df.corr(method="pearson")

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(columns='x', index='y', values='ppscore')

    pps_score_stats = pps_matrix_raw.query("ppscore < 1").filter(['ppscore']).describe().T
    print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
    print(pps_score_stats.round(3))

    return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix, CorrThreshold, PPS_Threshold,
                      figsize=(20, 12), font_annot=8):

    print("\n")
    print("* Analyse how the target variable for your ML models are correlated with other variables (features and target)")
    print("* Analyse multi-colinearity, that is, how the features are correlated among themselves")

    print("\n")
    print("*** Heatmap: Spearman Correlation ***")
    print("It evaluates monotonic relationship \n")
    heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Pearson Correlation ***")
    print("It evaluates the linear relationship between two continuous variables \n")
    heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Power Predictive Score (PPS) ***")
    print(f"PPS detects linear or non-linear relationships between two columns.\n"
          f"The score ranges from 0 (no predictive power) to 1 (perfect predictive power) \n")
    heatmap_pps(df=pps_matrix, threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot)

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

In [ ]:
DisplayCorrAndPPS(df_corr_pearson = df_corr_pearson,
                  df_corr_spearman = df_corr_spearman,
                  pps_matrix = pps_matrix,
                  CorrThreshold = 0.3, PPS_Threshold = 0.2,
                  figsize=(12,10), font_annot = 10)

---

# Data Cleaning

## Assessing Missing Data Levels

The following custom function was taken from CI walkthrough project 2. Its purpose is to list the variables with missing data with more details in relation to the missing data values and overall percentage in comparison to the whole data set. 

In [ ]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute/len(df)*100, 2)
    df_missing_data = (pd.DataFrame(
        data={"RowsWithMissingData": missing_data_absolute,
              "PercentageOfDataset": missing_data_percentage,
              "DataType": df.dtypes}
    )
                       .sort_values(by=['PercentageOfDataset'], ascending=False)
                       .query("PercentageOfDataset > 0")
                      )
    return df_missing_data

EvaluateMissingData(df)

## Data Cleaning Spreadsheet Summary

After reviewing the missing data levels and analysis of variables in the context of the business case, we want to preserve as much of it as possible and instead of dropping data instances or deleting variables from the data set. 

The following approaches will be tried to deal with missing data. A spreadsheet was used to record potential data cleaning approaches during development Heritage Housing Spreadsheet

| Variables | Observations | Possible Approach |
| :---------:| :---------: | :--------: |
| 'EnclosedPorch' | About 90% of data missing. The reason could be the non existence of a enclosed porch or data may not have been inputed due to user error | DROP |
| 'WoodDeckSF' | Also has a large portion of missing data (89.38%). The reason could be the non existence of a wood deck or data may not have been inputed due to user error | DROP |
| '2ndFlrSF' | There is a possibility that some houses do not have a second floor - 53.5% of data is a zero. Therefore one may assume that imputing the null values with 0 would add value.   | Arbitrary Number Imputation |
| 'MasVnrArea' | Most values for this variable are 0 indicating that there is no masonry veneer. Accordingly, we can assume that missing variables indicate the absence of a masonry veneer, so we can impute 0 as a value here. | Arbitrary Number Imputation |
| 'BedroomAbvGr'| The data have a mean of 2.9, a median of 3 and a std of 0.8. Noted that the distribution look fairly close to the normal distribution| Median Imputation |
| 'GarageYrBlt' | Data could be missing for some houses that do not have garages, therefore this data may have intentionally been missed. Although we could just imput zero to add value, this would not be valid because we cannot imput a year to a garage that does not exist.  | Median Imputation or Drop|
| 'LotFrontage' |  By checking the distribution of this variable, we found that it has a slight positive skew and strong positive kurtosis. It is therefore not normally distributed and we should impute the median. | Median Imputation |
| 'GarageFinish' | A little over 10% of the data is missing because some entries are 0, meaning there is no garage. A few approaches should be consider here such as use the machine model to predict  the GarageFinish based on other available variables present but for now we will simply impute an arbitrary value of "unf" to represent unfinished.  | Categorical Imputation ('unf') |
| 'BsmtFinType1' | Similar reasons to those listed for 'GarageFinish'. For now, we will simply impute an arbitrary value of "unf" to represent unfinished.| Categorical Imputation ('unf') |

## Dealing with Missing Data

The function used to assess the effect of the cleaning process is taken from Code Institute's lessons on Feature Engineering Module:

In [ ]:
def DataCleaningEffect(df_original,df_cleaned,variables_applied_with_method):

  flag_count=1 # Indicate plot number
  
  # distinguish between numerical and categorical variables
  categorical_variables = df_original.select_dtypes(exclude=['number']).columns 

  # scan over variables, 
    # first on variables that you applied the method
    # if the variable is a numerical plot, a histogram if categorical plot a barplot
  for set_of_variables in [variables_applied_with_method]:
    print("\n=====================================================================================")
    print(f"* Distribution Effect Analysis After Data Cleaning Method in the following variables:")
    print(f"{set_of_variables} \n\n")
  

    for var in set_of_variables:
      if var in categorical_variables:  # it is categorical variable: barplot
        
        df1 = pd.DataFrame({"Type":"Original","Value":df_original[var]})
        df2 = pd.DataFrame({"Type":"Cleaned","Value":df_cleaned[var]})
        dfAux = pd.concat([df1, df2], axis=0)
        fig , axes = plt.subplots(figsize=(15, 5))
        sns.countplot(hue='Type', data=dfAux, x="Value",palette=['#432371',"#FAAE7B"])
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.xticks(rotation=90)
        plt.legend() 

      else: # it is numerical variable: histogram

        fig , axes = plt.subplots(figsize=(10, 5))
        sns.histplot(data=df_original, x=var, color="#432371", label='Original', kde=True,element="step", ax=axes)
        sns.histplot(data=df_cleaned, x=var, color="#FAAE7B", label='Cleaned', kde=True,element="step", ax=axes)
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.legend() 

      plt.show()
      flag_count+= 1

### Data Cleaning Summary

* Drop : ['EnclosedPorch', 'WoodDeckSF', 'GarageYrBlt']
* Arbitrary Number Imputation : ['2ndFlrSF', 'MasVnrArea']
* MeanMedian Imputation : ['BedroomAbvGr', 'LotFrontage']
* Categorical Imputation : ['GarageFinish', 'BsmtFinType1']

### Drop Variables

In [ ]:
from feature_engine.selection import DropFeatures

variables_method = ['EnclosedPorch', 'WoodDeckSF', 'GarageYrBlt' ]
variables_method

imputer = DropFeatures(features_to_drop=variables_method)
df_method = imputer.fit_transform(df)
df_method.head(3)

### Arbitrary Number Imputation

In [ ]:
from feature_engine.imputation import ArbitraryNumberImputer

variables_method = ['2ndFlrSF', 'MasVnrArea']
variables_method

imputer = ArbitraryNumberImputer(arbitrary_number=0, variables=variables_method)
df_method = imputer.fit_transform(df)

DataCleaningEffect(df_original=df,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_method)

### MeanMedian Imputation

In [ ]:
variables_method = ['BedroomAbvGr', 'LotFrontage']
variables_method


imputer = MeanMedianImputer(imputation_method='median', variables=variables_method)
                            
df_method = imputer.fit_transform(df)

DataCleaningEffect(df_original=df,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_method)

### Categorical Imputation

In [ ]:

from feature_engine.imputation import CategoricalImputer

variables_method = ['GarageFinish', 'BsmtFinType1']
variables_method

imputer = CategoricalImputer(imputation_method='missing', fill_value='Unf', variables=variables_method)
df_method = imputer.fit_transform(df)

DataCleaningEffect(df_original=df,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_method)

## Train and Test Split

* Next we split the data into train and test sets

In [ ]:
from sklearn.model_selection import train_test_split

TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['SalePrice'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

* Now we apply the changes from the various imputation methods to the train and test sets.

In [ ]:
# Drop Variables 
variables_method = ['EnclosedPorch', 'WoodDeckSF', 'GarageYrBlt']
imputer = DropFeatures(features_to_drop=variables_method)
imputer.fit(TrainSet)
TrainSet, TestSet = imputer.transform(TrainSet), imputer.transform(TestSet)

# Arbitrary Number Imputer 
variables_method = ['2ndFlrSF', 'MasVnrArea']
imputer = ArbitraryNumberImputer(arbitrary_number=0, variables=variables_method)
imputer.fit(TrainSet)
TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

# MeanMedian Imputer
variables_method = ['BedroomAbvGr', 'LotFrontage']
imputer = MeanMedianImputer(imputation_method='median', variables=variables_method)
imputer.fit(TrainSet)
TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

# Categorical Imputation
variables_method = ['BsmtFinType1', 'GarageFinish']
imputer = CategoricalImputer(imputation_method='missing', fill_value='Unf', variables=variables_method)
imputer.fit(TrainSet)
TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)


* Evaluate missing data one more time

In [ ]:
df_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

### Data Cleaning Pipeline

In [ ]:
data_cleaning_pipeline = Pipeline([
      ( 'DropFeatures', DropFeatures(features_to_drop=['EnclosedPorch', 'WoodDeckSF', 'GarageYrBlt']) ),
      ( 'ArbitraryNumberImputer', ArbitraryNumberImputer(arbitrary_number=0, variables=['2ndFlrSF', 'MasVnrArea']) ),
      ( 'MeanMedianImputer', MeanMedianImputer(imputation_method='median', variables=['BedroomAbvGr' , 'LotFrontage']) ),
      ( 'CategoricalImputer', CategoricalImputer(imputation_method='missing', fill_value='Unf', variables=['BsmtFinType1', 'GarageFinish']) ),
      
])

df = data_cleaning_pipeline.fit_transform(df)
df.head()

---

# Push files to Repo

* If you do not need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
try:
  # create here your folder
  os.makedirs(name='outputs/datasets/cleaned')
except Exception as e:
  print(e)

# Save the Train Set
TrainSet.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)

# Save the Test Set
TestSet.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv", index=False)